## Problem Statement
Developing a Medical Research Assistant Using Retrieval-Augmented Generation (RAG)

#### Objective:
To create a Retrieval-Augmented Generation (RAG) application that functions as a medical research assistant, capable of answering complex medical questions using information retrieved from medical encyclopedia.

In [1]:
import chromadb

from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import numpy as np
from pypdf import PdfReader
from tqdm import tqdm
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)


In [2]:
from pypdf import PdfReader

reader = PdfReader("Encyclopedia of Medicine.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

individuals are highlighted as sidebar biographies thataccompany the
main topical essays. Articles follow astandardized format that provides
information at aglance. Rubrics include:
Disorders/Conditions
Tests/Treatments
Definition Definition
Description Purpose
Causes and
symptoms Precautions
Diagnosis DescriptionTreatment
Preparation
Alternative treatment Aftercare
Prognosis Risks
Prevention
Normal/Abnormal results
Resources ResourcesKey terms Key terms
In
recent years there has been a resurgence of interest
in holistic
medicine that emphasizes the connectionbetween mind and body. Aimed at
achieving and main-taining good health rather than just eliminating
disease,this approach has come to be known as alternative medi-
cine.
The Gale Encyclopedia of Medicine 2 includes a

Total chunks: 999


In [4]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

chantability or fitness for a particular purpose, nor does itguarantee
the accuracy, comprehensiveness, or timelinessof the information
contained in this product. readersshould be aware that the universe of
medical knowledgeis constantly growing and changing, and that
differencesof medical opinion exist among authorities. readers arealso
advised to seek professional diagnosis and treatmentfor any medical
condition, and to discuss informationobtained from this book with their
health care provider. gale encyclopedia of medicine 2 viiplease read —
important information

Total chunks: 1059


In [5]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
#print(embedding_function([token_split_texts[10]]))

In [6]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("info_medical", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

1059

In [7]:
# 
query = "What is PCR ?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

sure the presence of viral nucleic acids in the patient ’ sblood even
when there is no detectable antibody to hiv. this test works by
amplifying the presence of hiv nucle - ic acids in a blood sample.
numerous copies of a gene aremade by separating the two strands of dna
containingthe gene segment, marking its location, using dna poly -
merase to make a copy, and then continuously replicatingthe copies. it
is questionable whether pcr will replacewestern blotting as the method
of confirming aids diag - noses. although pcr can detect the low number
of per - sons ( 1 % ) with hiv infections that have not yet generatedan
antibody response to the virus, the overwhelmingmajority of infected
persons will be detected by elisascreening within one to three months
of infection. inaddition, pcr testing is based on present knowledge
ofthe genetic sequences in hiv


is sometimes used to confirm elisa results instead ofwestern blotting.
an ifa test detects the presence ofhiv antibody in a sample of t

## Generation With Cohere LLM

In [25]:
import cohere
co = cohere.Client('YOUR API KEY') # Create your trial API key on Cohere.com and replace it with "YOUR API KEY"

In [26]:
def rag(query, retrieved_documents, model="command"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert medical research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from encyclopedia of medicine. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = co.chat(
        model=model,
        message="What is PCR ?",
        documents=messages
    )

    return response

In [27]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print((output.text))

Polymerase chain reaction (PCR) is a test performed to evaluate false-negative results to the elisa and western blot tests. This test works by amplifying the presence of HIV nucleic acids in a blood sample. Numerous copies of a gene are made by separating the two strands of DNA containing the gene segment, marking its location, using DNA polymerase to make a copy, and then continuously replicating the copies. This allows for the detection of the virus.


# Gradio UI

Gradio is an open-source Python library that allows you to quickly create user-friendly web interfaces for machine learning models, APIs, and data science workflows. With Gradio, you can build interactive demos and web applications with minimal code. It provides a simple way to create UI components such as text inputs, sliders, buttons, and more, which can be linked to your functions or machine learning models.

### Key Features of Gradio:
1. **Easy to Use**: Gradio is designed to be easy to use, requiring minimal code to create interactive interfaces.
2. **Real-time Interactions**: Allows users to interact with machine learning models or functions in real-time.
3. **Wide Range of Components**: Provides various input and output components such as textboxes, sliders, dropdowns, images, and more.
4. **Automatic Hosting**: Gradio automatically hosts your interface on a local server, and you can also share it with others via a public link.
5. **Integration with Notebooks**: Works seamlessly with Jupyter Notebooks and other Python environments.
6. **Customizability**: Offers customization options for the interface, including layout, styling, ao documentation](https://gradio.app/docs/).

In [1]:
### Here's a simple example to demonstrate how Gradio works:


import gradio as gr

def greet(name):
    return f"Hello, {name}!"

# Create a Gradio interface
iface = gr.Interface(fn=greet, 
                     inputs=gr.Textbox(lines=2, placeholder="Enter your name..."), 
                     outputs="text",
                     title="Greeting App",
                     description="Enter your name and get a greeting!")

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.45.2, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


### Explanation of the Example:
1. **Function Definition**: `greet` function takes a name as input and returns a greeting message.
2. **Interface Creation**: 
    - `gr.Interface` is used to create the interface.
    - `fn=greet`: Specifies the function to be called.
    - `inputs`: Defines the input component (a textbox in this case).
    - `outputs`: Defines the output component (text in this case).
    - `title` and `description`: Provide a title and description for the interface.
3. **Launching the Interface**: `iface.launch()` starts the interface, which will be hosted on a local server.

For more information, you can visit the [Gradio documentation](https://gradio.app/docs/).

# Medical Research Assistant UI

In [13]:
import cohere
import gradio as gr
import chromadb  # Ensure this import is available for chroma_collection

# Initialize the Cohere client
co = cohere.Client('')  # This is your trial API key

# Assume chroma_collection is properly initialized
# chroma_collection = ...

# Define the RAG function
def rag(query, retrieved_documents, model="command"):
    information = "\n\n".join(retrieved_documents)
    print("Retrieved Documents: ", retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": ("You are a helpful expert medical research assistant. Your users are asking questions "
                        "about information contained in an encyclopedia of medicine. You will be shown the user's question, "
                        "and the relevant information from encyclopedia of medicine. Answer the user's question "
                        "using only this information.")
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = co.chat(
        model=model,
        documents=messages,
        message=query, # Corrected to 'messages' instead of 'message'
    )

    return response.text  # Adjust to extract the text from the response

# Define the chatbot function
def chatbot(query):
    try:
        # Query Chroma to get retrieved documents
        results = chroma_collection.query(query_texts=[query], n_results=5)
        retrieved_documents = results['documents'][0]
        
        # Debug: print retrieved documents
        #print("Retrieved Documents: ", retrieved_documents)
        
        # Call the RAG function
        response = rag(query, retrieved_documents)
        source_text = "\n\n".join(retrieved_documents)  # Combine the retrieved documents as source text
        return response, source_text
    except Exception as e:
        # Debug: print exception details
        print("Error: ", e)
        return str(e), ""

# Set up the Gradio interface
iface = gr.Interface(
    fn=chatbot,  # Pass the user query to the chatbot function
    inputs=gr.Textbox(lines=2, placeholder="Ask a medical question..."),
    outputs=[
        gr.Textbox(label="Response", lines=4),
        gr.Textbox(label="Source Text", lines=10)
    ],
    title="Medical Research Assistant",
    description="Ask any medical-related question, and I will provide answers based on the relevant information."
)

# Launch the Gradio interface
iface.launch()


Running on local URL:  http://127.0.0.1:7881
IMPORTANT: You are using gradio version 3.45.2, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.
